In [ ]:
# import disarray
!pip install disarray

row_sample_rate = 0.1

In [13]:
!pip install reload

     |████████████████████████████████| 75 kB 799 kB/s eta 0:00:01
  Created wheel for reload: filename=reload-0.9-py3-none-any.whl size=2879 sha256=1c3a9ef28f28b810ba8324bb4aca4228d7919b2883ee5508f368e7a744105d69
  Stored in directory: /home/jovyan/.cache/pip/wheels/63/5f/48/872ade1d79012c6ddca444710806bbecee30c0b086ee0183e9
Successfully built reload


In [16]:
import numpy as np
import pandas as pd
import time
import reload

from helpers import clone_drop_and_convert, clone_impute_data, perf_measures, round_decision, reduce_dataframe_using_min_non_null, get_coefficent_weights
from models import apply_linear_regression_model

# Developer note: If you modify the modules above, the changes to those modules will not be reflected until you 
# re-execute this cell, specifically the reload commands
# TODO this doesn't work yet
# reload(clone_drop_and_convert)
# reload(models)

pd.options.display.max_rows = 2000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 255

script_start = time.time()

raw_df = pd.read_csv('./assets/survey_results_public.csv')
schema = pd.read_csv('./assets/survey_results_schema.csv')

DUMMY_NA = True
FILL="Mean"
my_hand_picked_variables = [
    'Country',
    'Age',
    'Age1stCode',
    'ConvertedComp',
    'Employment',
    'JobSat',
    'JobSeek',
    'MainBranch',
    'NEWEdImpt',
    'OpSys',
    'OrgSize',
    'UndergradMajor',
    'WorkWeekHrs',
    'YearsCode',
    'YearsCodePro'
]

In [ ]:
df = clone_drop_and_convert(raw_df)

full_country = df.sample(frac=row_sample_rate, axis=0)
country_subset = df.query("Country in ['United States', 'Canada', 'United Kingdom', 'Australia', 'New Zealand']").sample(frac=row_sample_rate, axis=0)

full_country.shape
# (6446, 57)

country_subset.shape
# (2012, 57)

start = time.time()
full_country_imputed = clone_impute_data(full_country, dummy_na=True, fill_technique=FILL)
end = time.time()
print('full_country impute time %s', end - start)

start = time.time()
country_subset_imputed = clone_impute_data(country_subset, dummy_na=True, fill_technique=FILL)
end = time.time()
print('country_subset impute time %s', end - start)


# my_picked_variables = df.query("Country in ['United States', 'Canada', 'United Kingdom', 'Australia', 'New Zealand']")[my_hand_picked_variables]



In [ ]:
#cutoffs here pertains to the number of missing values allowed in the used columns.
#Therefore, lower values for the cutoff provides more predictors in the model.

# cutoffs = [5000, 3500, 2500, 1000, 100, 50, 30, 25, 0]
cutoffs = [10000, 5000, 2500, 1000, 500, 250]
is_usa = lambda country: 1 if country == 'United States' else 0

results = []
y_column = 'is_usa'

country_subset_imputed['is_usa'] = country_subset_imputed['Country'].map(is_usa)
y = country_subset_imputed[y_column]

print("country subset dataset has %s rows and %s columns" % (country_subset_imputed.shape[0], country_subset_imputed.shape[1]))
for cutoff in cutoffs:
    reduced_x = reduce_dataframe_using_min_non_null(country_subset_imputed, cutoff)
    X_columns = [ elem for elem in reduced_x.columns if elem not in ['Country', 'is_usa' ]]

    start = time.time()    
    X = reduced_x[X_columns]
    y = reduced_x[y_column]
    metrics, model = apply_linear_regression_model(X, y)
    end = time.time()
    metrics['model'] = model
    metrics['cutoff'] = cutoff
    metrics['X_columns'] = X_columns
    metrics['column_count'] = len(X_columns)
    metrics['duration'] = 1000 * (end - start)
    metrics['row_label'] = 'country_subset'
    metrics['column_label'] = 'all'
    metrics['row_sample_rate'] = row_sample_rate
    metrics['coefficients'] = get_coefficent_weights(model.coef_, X_columns)    
    results.append(metrics)

    print("a cutoff of min_values=%s yields %s columns and duration %ss" % (cutoff, len(X_columns), round(end - start,3)))

full_country_imputed['is_usa'] = full_country_imputed['Country'].map(is_usa)
y = full_country_imputed[y_column]
    
    
print("full country dataset has %s rows and %s columns" % (full_country_imputed.shape[0], full_country_imputed.shape[1]))
for cutoff in cutoffs:
    reduced_x = reduce_dataframe_using_min_non_null(full_country_imputed, cutoff)
    X_columns = [ elem for elem in reduced_x.columns if elem not in ['Country', 'is_usa' ]]

    start = time.time()    
    X = reduced_x[X_columns]
    y = reduced_x[y_column]
    metrics, model = apply_linear_regression_model(X, y)
    end = time.time()
    metrics['model'] = model
    metrics['cutoff'] = cutoff
    metrics['X_columns'] = X_columns
    metrics['column_count'] = len(X_columns)
    metrics['duration'] = 1000 * (end - start)
    metrics['row_label'] = 'full_country'
    metrics['column_label'] = 'all'
    metrics['row_sample_rate'] = row_sample_rate
    metrics['coefficients'] = get_coefficent_weights(model.coef_, X_columns)
    results.append(metrics)

    print("a cutoff of min_values=%s yields %s columns and duration %ss" % (cutoff, len(X_columns), round(end - start,3)))

# print('my_picked_variables')
# my_picked_variables = clone_impute_data(my_picked_variables, dummy_na=True, fill_technique=FILL)
# my_picked_variables['is_usa'] = my_picked_variables['Country'].map(is_usa)
# y = my_picked_variables[y_column]
# X_columns = [ elem for elem in my_picked_variables.columns if elem not in ['Country', 'is_usa' ]]    
# start = time.time()    
# X = my_picked_variables[X_columns]
# y = my_picked_variables[y_column]

# results.append(apply_linear_regression_model(X, y))
# end = time.time()
# print("my_picked_variables duration %s" % (end - start))


In [ ]:
result_df = pd.DataFrame(data=results).set_index(['row_label', 'cutoff', 'column_label'])
for to_round in ['TPR', 'TNR', 'PPV', 'NPV', 'ACC', 'duration']:
    result_df[to_round] = result_df[to_round].map(lambda x: round(x,3))

result_df.drop(columns='X_columns').sort_values('ACC')

In [ ]:
script_end = time.time()
print('script duration %s', end - start)

In [ ]:
result_df